In [1]:
from langchain_community.document_loaders import JSONLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms.llamacpp import LlamaCpp
from langchain_core.prompts import ChatPromptTemplate
import os

In [2]:
use_n_docs = 10

In [3]:
loader = JSONLoader("out.json", f"(.[:{use_n_docs}] | map({{rawtext}}))[].rawtext" if use_n_docs > 0 else ".[].rawtext")
docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

In [5]:
embed_model = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-large")

/home/jol0z0/code/poc_rag_blv/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with mean pooling.
/home/jol0z0/code/poc_rag_blv/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
%%time
vectorstore = Chroma.from_documents(documents=splits, embedding=embed_model, persist_directory="vectors", )

CPU times: user 9min 18s, sys: 1min 2s, total: 10min 20s
Wall time: 2min 35s


In [7]:
llm = LlamaCpp(model_path="../Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf", verbose=False, n_threads=4, n_ctx=8192)

In [8]:
prompt = ChatPromptTemplate([
    ("human", """<|begin_of_text|><|start_header_id|>system<|end_header_id|>Tu es un assistant possédant des connaissances sur les lois et règlements de la Base Législative Vaudoise, tu peux utiliser le contexte donné pour répondre aux questions. Réponds aux questions précisemment en 3 phrases. Si la question n'a pas de lien avec le contexte, ne mentionne pas l'existence du contexte.
<|eot_id|><|start_header_id|>user<|end_header_id|>{question}
Contexte: {context}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""")
])

In [9]:
rag = (
    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
%%time
rag.invoke("Quel montant d'aide mensuel peut demander un étudiant en HEP?")

CPU times: user 16min 17s, sys: 813 ms, total: 16min 18s
Wall time: 2min 19s


"L'étudiant peut demander une aide mensuelle de Fr. 1'200.-, étant donné qu'il s'agit d'un montant forfaitaire prévu par l'article 145 du code des lois de la Base Législative Vaudoise."